In [252]:
import json

In [253]:
# Read the template file
with open('config/open_api_template.json', 'r', encoding='utf-8') as file:
    swagger_template = json.load(file)

In [254]:
# Path to metadata file to be converted
metadata_path = 'in/VacancyCompetencyRequirement__c.json'

In [255]:


# Define the type mapping:
map_types = {
    'address': 'object',
    'boolean': 'boolean',
    'currency': 'number',
    'date': 'string',
    'datetime': 'string',
    'double': 'number',
    'id': 'string',
    'int': 'integer',
    'phone': 'string',
    'picklist': 'string',
    'reference': 'string',
    'string': 'string',
    'textarea': 'string',
    'url': 'string'
 }

# Schema template
sobject_template = {
    "type": "object",
    "properties": {
      "apiSObjectType": {
        "type": "string",
        "description": "The type of the current sObject, used as a discriminator for OAS tooling."
      },
      "attributes": {
        "type": "object",
        "properties": {
          "type": {
            "type": "string"
          },
          "url": {
            "type": "string"
          }
        },
        "readOnly": True
      }
    },
    "discriminator": {
      "propertyName": "apiSObjectType"
    }
  }

# Read the metadata file
with open(metadata_path, 'r', encoding='utf-8') as file:
    metadata = json.load(file)



In [256]:
unique_types = { obj['type'] for obj in metadata['fields'] }
map_types_set = set(map_types.keys())
check = unique_types - map_types_set
check

set()

In [257]:
name = metadata['name']

# Build the properties
properties = {}
for field in metadata['fields']:
    property = {}
    property['type'] = map_types[field['type']]
    if not field['updateable']:
        property['readOnly'] = True
    if field['type'] == 'date':
        property['format'] = 'date'
    elif field['type'] == 'datetime':
        property['format'] = 'date-time'
    elif field['type'] == 'int':
        property['format'] = 'int32'

    # Save the property
    properties[field['name']] = property


In [258]:
# Initialise the swagger
swagger = swagger_template

In [259]:
# Add the new QueryResult schema
query_name = 'QueryResult' + name
query_result_schema = swagger['components']['schemas']['QueryResult']
query_result_schema['properties']['records']['items']['properties'] = properties

In [260]:
# Add the new SObject schema
sobject_name = 'SObject' + name
sobject_schema = sobject_template
sobject_schema['properties'].update(properties)


In [261]:
# Add the new schemas
swagger['components']['schemas'][query_name] = query_result_schema
swagger['components']['schemas'][sobject_name] = sobject_schema

# Update the query path
swagger['paths']['/query']['get']['operationId'] = swagger['paths']['/query']['get']['operationId'] + name
swagger['paths']['/query']['get']['responses']['200']['content']['application/xml']['schema']['$ref'] = '#/components/schemas/' + query_name
swagger['paths']['/query']['get']['responses']['200']['content']['application/json']['schema']['$ref'] = '#/components/schemas/' + query_name

# Update the create path
path_name = '/sobjects/' + name
action = 'post'
swagger['paths'][path_name] = swagger['paths'].pop('/sobjects/{sObject}')
del swagger['paths'][path_name]['get']
swagger['paths'][path_name][action]['operationId'] = swagger['paths'][path_name][action]['operationId'] + name
swagger['paths'][path_name][action]['parameters'] = [obj for obj in swagger['paths'][path_name][action]['parameters'] if obj['name'] != 'sObject']
del swagger['paths'][path_name][action]['requestBody']['content']['application/json']['examples']
swagger['paths'][path_name][action]['requestBody']['content']['application/json']['schema']['$ref'] = '#/components/schemas/' + sobject_name

# Update the get object path
path_name = '/sobjects/' + name + '/{id}'
swagger['paths'][path_name] = swagger['paths'].pop('/sobjects/{sObject}/{id}')
action = 'get'
swagger['paths'][path_name][action]['operationId'] = swagger['paths'][path_name][action]['operationId'] + name
swagger['paths'][path_name][action]['parameters'] = [obj for obj in swagger['paths'][path_name][action]['parameters'] if obj['name'] != 'sObject']
swagger['paths'][path_name][action]['responses']['200']['content']['application/json']['schema']['$ref'] = '#/components/schemas/' + sobject_name

# Update the delete object path
action = 'delete'
swagger['paths'][path_name][action]['operationId'] = swagger['paths'][path_name][action]['operationId'] + name
swagger['paths'][path_name][action]['parameters'] = [obj for obj in swagger['paths'][path_name][action]['parameters'] if obj['name'] != 'sObject']

# Update the patch object path
action = 'patch'
swagger['paths'][path_name][action]['operationId'] = swagger['paths'][path_name][action]['operationId'] + name
swagger['paths'][path_name][action]['parameters'] = [obj for obj in swagger['paths'][path_name][action]['parameters'] if obj['name'] != 'sObject']
del swagger['paths'][path_name][action]['requestBody']['content']['application/json']['examples']
swagger['paths'][path_name][action]['requestBody']['content']['application/json']['schema']['$ref'] = '#/components/schemas/' + sobject_name


In [262]:
with open(f'out/Salesforce_{ name }.json', 'w', encoding='utf-8') as file:
    json.dump(swagger, file, indent=4)